# Signal Flow & Training Dynamics in SOEN Model

This notebook visualizes how signals propagate through the minimal SOEN model and how gradients flow during training.

## Model Architecture
```
Input x → [J_in] → φ → SingleDendrite → s → [J_out] → Output y
```

## Key Equations

### 1. Input Flux Computation
$$\phi(t) = J_{in} \cdot x(t) + \phi_{offset}$$

### 2. SingleDendrite Dynamics (ODE)
$$\frac{ds}{dt} = \gamma^+ \cdot g(\phi) - \gamma^- \cdot s$$

where $g(\phi)$ is the source function (e.g., Heaviside fit)

### 3. Output Computation
$$y = J_{out} \cdot s_{final}$$

### 4. Loss Function
$$\mathcal{L} = \frac{1}{N} \sum_{i=1}^{N} (y_i - y_{target,i})^2$$

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from soen_toolkit.core import (
    ConnectionConfig,
    LayerConfig,
    SimulationConfig,
    SOENModelCore,
)

torch.manual_seed(42)
np.random.seed(42)

## 1. Build Model with Tracking Enabled

We enable `track_phi=True` to record the input flux at each timestep.

In [ ]:
def build_model_with_tracking(j_in=0.15, j_out=1.5):
    """Build SOEN model with phi tracking enabled."""
    sim_cfg = SimulationConfig(
        dt=50.0,
        input_type="state",
        track_phi=True,  # Track input flux!
        track_power=False,
    )
    
    layer0 = LayerConfig(
        layer_id=0,
        layer_type="Input",
        params={"dim": 1},
    )
    
    layer1 = LayerConfig(
        layer_id=1,
        layer_type="SingleDendrite",
        params={
            "dim": 1,
            "solver": "FE",
            "source_func": "Heaviside_fit_state_dep",
            "phi_offset": 0.02,
            "bias_current": 1.98,
            "gamma_plus": 0.0005,
            "gamma_minus": 1e-6,
            "learnable_params": {
                "phi_offset": False,
                "bias_current": False,
                "gamma_plus": False,
                "gamma_minus": False,
            },
        },
    )
    
    layer2 = LayerConfig(
        layer_id=2,
        layer_type="Input",
        params={"dim": 1},
    )
    
    conn01 = ConnectionConfig(
        from_layer=0, to_layer=1,
        connection_type="all_to_all",
        learnable=True,
        params={"init": "constant", "value": j_in},
    )
    
    conn12 = ConnectionConfig(
        from_layer=1, to_layer=2,
        connection_type="all_to_all",
        learnable=True,
        params={"init": "constant", "value": j_out},
    )
    
    model = SOENModelCore(
        sim_config=sim_cfg,
        layers_config=[layer0, layer1, layer2],
        connections_config=[conn01, conn12],
    )
    
    return model

# Build model
J_IN = 0.15
J_OUT = 1.5
model = build_model_with_tracking(J_IN, J_OUT)

print("Trainable parameters:")
for name, p in model.named_parameters():
    if p.requires_grad:
        print(f"  {name}: {p.item():.4f}")

## 2. Create Test Input Signal

We'll use a simple constant input to clearly see the dynamics.

In [ ]:
# Create input: constant value over time
SEQ_LEN = 50
INPUT_VALUE = 0.15  # Constant input flux

# Single sample, constant input
x_input = torch.full((1, SEQ_LEN, 1), INPUT_VALUE)

print(f"Input shape: {x_input.shape}")
print(f"Input value: {INPUT_VALUE}")

## 3. Visualize Complete Signal Flow

Let's trace the signal through every stage of the model:

```
x(t) → φ(t) = J_in·x(t) + φ_offset → s(t) via ODE → y = J_out·s_final
```

In [ ]:
# Run forward pass
model.eval()
with torch.no_grad():
    final_hist, all_histories = model(x_input)

# Extract signals
x_signal = x_input[0, :, 0].numpy()  # Input signal
s_history = all_histories[1][0, :, 0].numpy()  # SingleDendrite state over time
output_history = all_histories[2][0, :, 0].numpy()  # Output layer state

# Get phi (input flux to SingleDendrite) if tracked
phi_history = None
if hasattr(model.layers[1], 'phi_history') and model.layers[1].phi_history is not None:
    phi_history = model.layers[1].phi_history[0, :, 0].numpy()

# Compute phi manually for visualization
phi_offset = 0.02
phi_computed = J_IN * x_signal + phi_offset

# Time axis
t = np.arange(SEQ_LEN)
t_state = np.arange(len(s_history))  # State has +1 for initial condition

print(f"Input x: {INPUT_VALUE}")
print(f"Computed φ = J_in·x + φ_offset = {J_IN}·{INPUT_VALUE} + {phi_offset} = {J_IN * INPUT_VALUE + phi_offset:.4f}")
print(f"Final state s: {s_history[-1]:.4f}")
print(f"Output y = J_out·s = {J_OUT}·{s_history[-1]:.4f} = {J_OUT * s_history[-1]:.4f}")

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(16, 12))
gs = GridSpec(3, 3, figure=fig, hspace=0.3, wspace=0.3)

# === Row 1: Signal Flow ===

# 1a. Input Signal x(t)
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t, x_signal, 'b-', linewidth=2, label='x(t)')
ax1.fill_between(t, 0, x_signal, alpha=0.3)
ax1.set_xlabel('Time step')
ax1.set_ylabel('x(t)')
ax1.set_title('① Input Signal x(t)', fontsize=12, fontweight='bold')
ax1.set_ylim(0, 0.25)
ax1.grid(True, alpha=0.3)
ax1.legend()

# 1b. Input Flux φ(t) = J_in·x(t) + φ_offset
ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(t, phi_computed, 'g-', linewidth=2, label=f'φ = {J_IN}·x + {phi_offset}')
ax2.axhline(y=phi_offset, color='gray', linestyle='--', label=f'φ_offset = {phi_offset}')
ax2.fill_between(t, phi_offset, phi_computed, alpha=0.3, color='green', label=f'J_in·x contribution')
ax2.set_xlabel('Time step')
ax2.set_ylabel('φ(t)')
ax2.set_title('② Input Flux φ(t) = J_in·x + φ_offset', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=8)

# 1c. SingleDendrite State s(t)
ax3 = fig.add_subplot(gs[0, 2])
ax3.plot(t_state, s_history, 'r-', linewidth=2, label='s(t)')
ax3.axhline(y=s_history[-1], color='red', linestyle='--', alpha=0.5, label=f's_final = {s_history[-1]:.3f}')
ax3.fill_between(t_state, 0, s_history, alpha=0.3, color='red')
ax3.set_xlabel('Time step')
ax3.set_ylabel('s(t)')
ax3.set_title('③ SingleDendrite State s(t)', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.legend()

# === Row 2: Parameter Effects ===

# 2a. Effect of J_in on φ
ax4 = fig.add_subplot(gs[1, 0])
j_in_values = [0.05, 0.10, 0.15, 0.20, 0.25]
colors = plt.cm.Blues(np.linspace(0.3, 1, len(j_in_values)))
for j_in, color in zip(j_in_values, colors):
    phi_val = j_in * INPUT_VALUE + phi_offset
    ax4.axhline(y=phi_val, color=color, linewidth=2, label=f'J_in={j_in:.2f} → φ={phi_val:.3f}')
ax4.set_ylabel('φ (input flux)')
ax4.set_title('Effect of J_in on Input Flux φ', fontsize=12, fontweight='bold')
ax4.set_xlim(0, 1)
ax4.set_xticks([])
ax4.legend(loc='right', fontsize=8)
ax4.grid(True, alpha=0.3, axis='y')

# 2b. Effect of J_in on s(t) dynamics
ax5 = fig.add_subplot(gs[1, 1])
for j_in, color in zip(j_in_values, colors):
    test_model = build_model_with_tracking(j_in=j_in, j_out=J_OUT)
    test_model.eval()
    with torch.no_grad():
        _, test_histories = test_model(x_input)
    s_test = test_histories[1][0, :, 0].numpy()
    ax5.plot(s_test, color=color, linewidth=2, label=f'J_in={j_in:.2f}')
ax5.set_xlabel('Time step')
ax5.set_ylabel('s(t)')
ax5.set_title('Effect of J_in on State Evolution', fontsize=12, fontweight='bold')
ax5.legend(fontsize=8)
ax5.grid(True, alpha=0.3)

# 2c. Effect of J_out on output
ax6 = fig.add_subplot(gs[1, 2])
j_out_values = [0.5, 1.0, 1.5, 2.0, 2.5]
s_final = s_history[-1]
outputs = [j_out * s_final for j_out in j_out_values]
colors_out = plt.cm.Oranges(np.linspace(0.3, 1, len(j_out_values)))
bars = ax6.bar(range(len(j_out_values)), outputs, color=colors_out)
ax6.set_xticks(range(len(j_out_values)))
ax6.set_xticklabels([f'J_out={j:.1f}' for j in j_out_values], rotation=45)
ax6.set_ylabel('Output y = J_out · s_final')
ax6.set_title(f'Effect of J_out on Output (s_final={s_final:.3f})', fontsize=12, fontweight='bold')
ax6.grid(True, alpha=0.3, axis='y')
for bar, out in zip(bars, outputs):
    ax6.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{out:.2f}', ha='center', fontsize=9)

# === Row 3: Complete Pipeline Diagram ===
ax7 = fig.add_subplot(gs[2, :])
ax7.set_xlim(0, 10)
ax7.set_ylim(0, 3)
ax7.axis('off')
ax7.set_title('Complete Signal Flow Pipeline', fontsize=14, fontweight='bold', pad=20)

# Draw boxes and arrows
box_style = dict(boxstyle='round,pad=0.3', facecolor='lightblue', edgecolor='black', linewidth=2)
arrow_style = dict(arrowstyle='->', color='black', linewidth=2)

# Input
ax7.text(0.5, 2, f'Input\nx = {INPUT_VALUE}', ha='center', va='center', fontsize=11, bbox=box_style)

# Arrow with J_in
ax7.annotate('', xy=(1.8, 2), xytext=(1.1, 2), arrowprops=arrow_style)
ax7.text(1.45, 2.4, f'× J_in\n({J_IN})', ha='center', va='center', fontsize=10, color='blue', fontweight='bold')

# Phi computation
ax7.text(2.5, 2, f'+ φ_offset\n({phi_offset})', ha='center', va='center', fontsize=10, 
         bbox=dict(boxstyle='round', facecolor='lightgreen', edgecolor='black'))

# Arrow to phi
ax7.annotate('', xy=(3.8, 2), xytext=(3.2, 2), arrowprops=arrow_style)

# Phi result
phi_val = J_IN * INPUT_VALUE + phi_offset
ax7.text(4.5, 2, f'φ = {phi_val:.4f}', ha='center', va='center', fontsize=11, 
         bbox=dict(boxstyle='round', facecolor='lightyellow', edgecolor='black', linewidth=2))

# Arrow to ODE
ax7.annotate('', xy=(5.8, 2), xytext=(5.2, 2), arrowprops=arrow_style)

# SingleDendrite ODE
ax7.text(6.8, 2, 'SingleDendrite\nODE Solver', ha='center', va='center', fontsize=11,
         bbox=dict(boxstyle='round', facecolor='lightsalmon', edgecolor='black', linewidth=2))
ax7.text(6.8, 1.2, r'$\frac{ds}{dt} = \gamma^+ g(\phi) - \gamma^- s$', ha='center', va='center', fontsize=10)

# Arrow to s_final
ax7.annotate('', xy=(8.3, 2), xytext=(7.7, 2), arrowprops=arrow_style)

# s_final
ax7.text(8.8, 2, f's_final\n= {s_history[-1]:.4f}', ha='center', va='center', fontsize=11,
         bbox=dict(boxstyle='round', facecolor='lightcoral', edgecolor='black', linewidth=2))

# Arrow with J_out
ax7.annotate('', xy=(9.8, 2), xytext=(9.4, 2), arrowprops=arrow_style)
ax7.text(9.6, 2.4, f'× J_out\n({J_OUT})', ha='center', va='center', fontsize=10, color='orange', fontweight='bold')

# Output
output_val = J_OUT * s_history[-1]
ax7.text(10.3, 2, f'Output\ny = {output_val:.4f}', ha='center', va='center', fontsize=11, 
         bbox=dict(boxstyle='round', facecolor='plum', edgecolor='black', linewidth=2))

# Formula summary
ax7.text(5.5, 0.5, 
         f'Summary: y = J_out × s_final = {J_OUT} × {s_history[-1]:.4f} = {output_val:.4f}\n'
         f'where s_final comes from ODE driven by φ = J_in × x + φ_offset = {J_IN} × {INPUT_VALUE} + {phi_offset} = {phi_val:.4f}',
         ha='center', va='center', fontsize=11, 
         bbox=dict(boxstyle='round', facecolor='white', edgecolor='gray', alpha=0.8))

plt.tight_layout()
plt.show()

## 4. Gradient Flow During Training

During backpropagation, gradients flow backward through the computation graph:

```
∂L/∂J_out ← ∂L/∂y × ∂y/∂J_out
∂L/∂J_in  ← ∂L/∂y × ∂y/∂s × ∂s/∂φ × ∂φ/∂J_in
```

### Gradient Formulas:

**For J_out (simple):**
$$\frac{\partial \mathcal{L}}{\partial J_{out}} = \frac{\partial \mathcal{L}}{\partial y} \cdot s_{final}$$

**For J_in (through ODE):**
$$\frac{\partial \mathcal{L}}{\partial J_{in}} = \frac{\partial \mathcal{L}}{\partial y} \cdot J_{out} \cdot \frac{\partial s_{final}}{\partial \phi} \cdot x$$

The term $\frac{\partial s_{final}}{\partial \phi}$ requires backpropagation through the ODE solver (adjoint method or direct differentiation).

In [ ]:
# Demonstrate gradient computation
model = build_model_with_tracking(J_IN, J_OUT)
model.train()

# Target value
y_target = torch.tensor([[1.0]])  # We want output to be 1.0

# Forward pass
final_hist, all_histories = model(x_input)
y_pred = final_hist[:, -1, :]  # Take final timestep

print(f"Input x: {INPUT_VALUE}")
print(f"Predicted y: {y_pred.item():.4f}")
print(f"Target y: {y_target.item():.4f}")

# Compute loss
loss = nn.MSELoss()(y_pred, y_target)
print(f"\nLoss: {loss.item():.6f}")

# Backward pass
loss.backward()

# Show gradients
print("\n" + "="*50)
print("GRADIENTS:")
print("="*50)
for name, p in model.named_parameters():
    if p.requires_grad:
        print(f"\n{name}:")
        print(f"  Value:    {p.item():.6f}")
        print(f"  Gradient: {p.grad.item():.6f}")

In [ ]:
# Visualize gradient flow
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Get parameter info
params = {name: (p.item(), p.grad.item()) for name, p in model.named_parameters() if p.requires_grad}
param_names = list(params.keys())
values = [params[n][0] for n in param_names]
grads = [params[n][1] for n in param_names]

# Plot 1: Parameter values
ax1 = axes[0]
bars1 = ax1.bar(['J_in', 'J_out'], values, color=['blue', 'orange'])
ax1.set_ylabel('Parameter Value')
ax1.set_title('Current Parameter Values', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars1, values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{val:.4f}', ha='center', fontsize=11)

# Plot 2: Gradients
ax2 = axes[1]
colors = ['green' if g < 0 else 'red' for g in grads]
bars2 = ax2.bar(['∂L/∂J_in', '∂L/∂J_out'], grads, color=colors)
ax2.axhline(y=0, color='black', linewidth=0.5)
ax2.set_ylabel('Gradient Value')
ax2.set_title('Gradients (direction to INCREASE loss)', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')
for bar, grad in zip(bars2, grads):
    offset = 0.01 if grad >= 0 else -0.01
    va = 'bottom' if grad >= 0 else 'top'
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + offset, 
             f'{grad:.4f}', ha='center', va=va, fontsize=11)

plt.suptitle(f'Gradient Analysis (y_pred={y_pred.item():.3f}, y_target={y_target.item():.3f}, loss={loss.item():.4f})', 
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nInterpretation:")
for name, (val, grad) in zip(['J_in', 'J_out'], [(values[0], grads[0]), (values[1], grads[1])]):
    direction = "DECREASE" if grad > 0 else "INCREASE"
    print(f"  {name}: gradient = {grad:.4f} → optimizer will {direction} this parameter")

## 5. Training Dynamics Visualization

Let's watch how J_in and J_out evolve during training, and see how the signal flow changes.

In [ ]:
# Training with detailed tracking
def train_with_tracking(target_y, n_epochs=50, lr=0.1):
    """Train model and track all intermediate values."""
    model = build_model_with_tracking(j_in=0.1, j_out=0.5)  # Start far from optimal
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    history = {
        'j_in': [], 'j_out': [],
        'phi': [], 's_final': [], 'y_pred': [],
        'loss': [],
        'grad_j_in': [], 'grad_j_out': []
    }
    
    y_target = torch.tensor([[target_y]])
    
    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()
        
        # Get current parameters
        params = list(model.parameters())
        j_in = params[0].item()
        j_out = params[1].item()
        
        # Forward
        final_hist, all_hist = model(x_input)
        y_pred = final_hist[:, -1, :]
        s_final = all_hist[1][0, -1, 0].item()
        phi = j_in * INPUT_VALUE + 0.02
        
        # Loss & backward
        loss = criterion(y_pred, y_target)
        loss.backward()
        
        # Record
        history['j_in'].append(j_in)
        history['j_out'].append(j_out)
        history['phi'].append(phi)
        history['s_final'].append(s_final)
        history['y_pred'].append(y_pred.item())
        history['loss'].append(loss.item())
        history['grad_j_in'].append(params[0].grad.item())
        history['grad_j_out'].append(params[1].grad.item())
        
        optimizer.step()
    
    return history

# Train to reach target output of 1.0
TARGET_Y = 1.0
history = train_with_tracking(TARGET_Y, n_epochs=100, lr=0.05)

print(f"Training complete!")
print(f"  Initial: J_in={history['j_in'][0]:.4f}, J_out={history['j_out'][0]:.4f}, y={history['y_pred'][0]:.4f}")
print(f"  Final:   J_in={history['j_in'][-1]:.4f}, J_out={history['j_out'][-1]:.4f}, y={history['y_pred'][-1]:.4f}")
print(f"  Target:  y={TARGET_Y}")

In [ ]:
# Comprehensive training visualization
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
epochs = range(len(history['loss']))

# 1. Loss curve
ax1 = axes[0, 0]
ax1.plot(epochs, history['loss'], 'k-', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('MSE Loss')
ax1.set_title('Training Loss', fontsize=12, fontweight='bold')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

# 2. J_in evolution
ax2 = axes[0, 1]
ax2.plot(epochs, history['j_in'], 'b-', linewidth=2, label='J_in')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('J_in')
ax2.set_title('J_in Evolution (Input Weight)', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. J_out evolution
ax3 = axes[0, 2]
ax3.plot(epochs, history['j_out'], 'orange', linewidth=2, label='J_out')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('J_out')
ax3.set_title('J_out Evolution (Output Weight)', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4. Signal flow: phi
ax4 = axes[1, 0]
ax4.plot(epochs, history['phi'], 'g-', linewidth=2)
ax4.set_xlabel('Epoch')
ax4.set_ylabel('φ = J_in·x + φ_offset')
ax4.set_title('Input Flux φ Evolution', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3)

# 5. Signal flow: s_final
ax5 = axes[1, 1]
ax5.plot(epochs, history['s_final'], 'r-', linewidth=2)
ax5.set_xlabel('Epoch')
ax5.set_ylabel('s_final')
ax5.set_title('SingleDendrite Final State Evolution', fontsize=12, fontweight='bold')
ax5.grid(True, alpha=0.3)

# 6. Output y vs target
ax6 = axes[1, 2]
ax6.plot(epochs, history['y_pred'], 'purple', linewidth=2, label='y_pred')
ax6.axhline(y=TARGET_Y, color='red', linestyle='--', linewidth=2, label=f'Target = {TARGET_Y}')
ax6.set_xlabel('Epoch')
ax6.set_ylabel('Output y')
ax6.set_title('Output y = J_out · s_final', fontsize=12, fontweight='bold')
ax6.legend()
ax6.grid(True, alpha=0.3)

plt.suptitle('Training Dynamics: How J_in and J_out are Learned', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Gradient evolution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(epochs, history['grad_j_in'], 'b-', linewidth=2)
ax1.axhline(y=0, color='black', linewidth=0.5)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('∂L/∂J_in')
ax1.set_title('Gradient of Loss w.r.t. J_in', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.plot(epochs, history['grad_j_out'], 'orange', linewidth=2)
ax2.axhline(y=0, color='black', linewidth=0.5)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('∂L/∂J_out')
ax2.set_title('Gradient of Loss w.r.t. J_out', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.suptitle('Gradient Evolution During Training', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nGradient Interpretation:")
print("  - When gradient is positive: optimizer decreases the parameter")
print("  - When gradient is negative: optimizer increases the parameter")
print("  - As training converges, gradients approach zero")

## 6. Summary: What J_in and J_out Control

### J_in (Input Weight)
- **Location**: Connection from Input layer to SingleDendrite
- **Formula**: φ = **J_in** · x + φ_offset
- **Effect**: Scales the input signal before it enters the neuron
- **Physical meaning**: Controls how strongly the input flux drives the neuron

### J_out (Output Weight)  
- **Location**: Connection from SingleDendrite to Output layer
- **Formula**: y = **J_out** · s_final
- **Effect**: Scales the neuron's final state to produce the output
- **Physical meaning**: Controls the readout gain from the neuron

### Training Process
1. **Forward pass**: x → φ → s(t) → y
2. **Loss computation**: L = (y - y_target)²
3. **Backward pass**: Compute ∂L/∂J_in and ∂L/∂J_out
4. **Update**: J_in -= lr · ∂L/∂J_in, J_out -= lr · ∂L/∂J_out

### Key Insight
With only J_in and J_out, the model can learn **scaling** relationships but NOT **affine** transformations (y = αx + β) because there's no learnable bias term.

In [ ]:
print("Notebook complete!")
print("\nKey takeaways:")
print("  1. J_in controls input scaling: φ = J_in·x + φ_offset")
print("  2. J_out controls output scaling: y = J_out·s_final")
print("  3. Gradients flow backward through the ODE solver")
print("  4. Both parameters adjust to minimize the loss")